In [1]:
import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [2]:
import numpy as np
import pickle

In [3]:
n_input = 1500
n_hidden1 = 1024
n_hidden2 = 512
n_hidden3 = 64
n_output = 6

In [4]:
learning_rate = 1e-4
n_iteration = 101
batch_size = 128
dropout = 0.1

In [5]:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
# keep_prob = tf.placeholder(tf.float32) # used to control the dropout rate

In [20]:
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1))
}

In [21]:
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

In [22]:
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
# layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.add(tf.matmul(layer_3, weights['out']), biases['out'])

In [23]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [24]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [25]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [26]:
def get_data(cat):
    with open(cat+'_agg_Y.pkl', 'rb') as fy:
        _Y = pickle.load(fy)

    with open(cat+'_agg_X.pkl', 'rb') as fx:
        _X = pickle.load(fx)
    return (_X, _Y)

In [27]:
dev_X, dev_Y = get_data('train')

In [28]:
def get_batch(idx, batch_size):
    lw = idx * batch_size
    hs = lw + batch_size
    if hs < len(dev_X):
        return (dev_X[lw:hs][:], dev_Y[lw:hs][:])
    else:
        return (dev_X[lw:][:], dev_Y[lw:][:])

In [29]:
n_batches = len(dev_X)//batch_size

In [30]:
# train on mini batches
for i in range(n_iteration):
    for j in range(n_batches+1):
        (batch_x, batch_y) = get_batch(j, batch_size)
        sess.run(train_step, feed_dict={X: batch_x, Y: batch_y})
        
        # print loss and accuracy (per minibatch)
    if i % 5 == 0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y})
        print("Iteration", str(i), "\t| Batch", str(j), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

Iteration 0 	| Batch 127 	| Loss = 0.89512557 	| Accuracy = 0.72727275
Iteration 5 	| Batch 127 	| Loss = 0.57541007 	| Accuracy = 0.7878788
Iteration 10 	| Batch 127 	| Loss = 0.54573834 	| Accuracy = 0.8080808
Iteration 15 	| Batch 127 	| Loss = 0.53418946 	| Accuracy = 0.8080808
Iteration 20 	| Batch 127 	| Loss = 0.52710944 	| Accuracy = 0.7979798
Iteration 25 	| Batch 127 	| Loss = 0.522704 	| Accuracy = 0.7979798
Iteration 30 	| Batch 127 	| Loss = 0.51908237 	| Accuracy = 0.7777778
Iteration 35 	| Batch 127 	| Loss = 0.5157165 	| Accuracy = 0.7878788
Iteration 40 	| Batch 127 	| Loss = 0.51239306 	| Accuracy = 0.7979798
Iteration 45 	| Batch 127 	| Loss = 0.5091702 	| Accuracy = 0.7979798
Iteration 50 	| Batch 127 	| Loss = 0.50617635 	| Accuracy = 0.8080808
Iteration 55 	| Batch 127 	| Loss = 0.50349355 	| Accuracy = 0.8080808
Iteration 60 	| Batch 127 	| Loss = 0.50112236 	| Accuracy = 0.8181818
Iteration 65 	| Batch 127 	| Loss = 0.4989967 	| Accuracy = 0.8181818
Iteration 70

In [31]:
test_X, test_Y = get_data('test')

test_accuracy = sess.run(accuracy, feed_dict={X: test_X, Y: test_Y})
print("\nAccuracy on test set : ", test_accuracy)


Accuracy on test set :  0.7513765
